In [ ]:
import json
import boto3
import random
import logging
from datetime import datetime

# Initialize the EventBridge client using IAM roles
eventbridge = boto3.client('events', region_name='us-east-1')

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Constants
EVENT_BUS_NAME = "orchestrationBus"  # Replace with your EventBridge bus name

# List of event names
EVENT_NAMES = [
    "RevokeSecurityGroupEgress",
    "LookupEvents",
    "SendHeartBeat",
    "DescribeInstances",
    "DeleteSecurityGroup",
    "RevokeSecurityGroupIngress",
    "PutCredentials",
    "GetEnvironmentStatus",
    "AuthorizeSecurityGroupIngress",
    "CreateSecurityGroup",
    "DescribeRegions",
    "GetEventSelectors",
    "ListTags",
    "GetInsightSelectors",
    "GetTrailStatus",
    "DescribeTrails",
    "CreateSession",
    "DeleteSession",
    "RedeemCode",
    "DescribeEnvironments",
    "ListTrails",
    "RunInstances",
    "TerminateInstances",
    "DescribeInstanceTypes",
    "DescribeImages",
    "GetInstanceMetadataDefaults",
    "GetEbsDefaultKmsKeyId",
    "GetEbsEncryptionByDefault",
    "DescribeSnapshots",
    "DescribeInstanceTypeOfferings",
    "DescribeVpcs",
    "DescribeSubnets",
    "DescribeAccountAttributes",
    "DescribeSecurityGroups",
    "DescribeTags",
    "DescribeInstanceStatus",
    "DescribeAddresses",
    "DescribeVolumeStatus",
    "DescribeHosts",
    "DescribeCapacityReservations",
    "DescribeVolumes",
    "DescribeAvailabilityZones",
    "DescribeLaunchTemplates",
    "DescribePlacementGroups",
    "ListEventDataStores",
    "SharedSnapshotVolumeCreated",
    "StartLogging",
    "CreateTrail",
    "PutEventSelectors",
    "AutomatedDefaultVpcCreation",
    # Add more events as needed
]

# Mapping of event names to sources
EVENT_SOURCES = {
    "myapp.ec2": [
        "RunInstances",
        "TerminateInstances",
        "DescribeInstances",
        "CreateSecurityGroup",
        "DeleteSecurityGroup",
        "AuthorizeSecurityGroupIngress",
        "RevokeSecurityGroupIngress",
        "RevokeSecurityGroupEgress",
        "DescribeInstanceTypes",
        "DescribeImages",
        "DescribeSnapshots",
        "DescribeVpcs",
        "DescribeSubnets",
        "DescribeSecurityGroups",
        "DescribeAddresses",
        "DescribeAvailabilityZones",
        "DescribeLaunchTemplates",
        "DescribePlacementGroups",
        "AutomatedDefaultVpcCreation",
        "DescribeRegions",
    ],
    "myapp.cloudtrail": [
        "LookupEvents",
        "GetEventSelectors",
        "ListTags",
        "GetInsightSelectors",
        "GetTrailStatus",
        "DescribeTrails",
        "ListTrails",
        "StartLogging",
        "CreateTrail",
        "PutEventSelectors",
        "ListEventDataStores",
        "GetEbsEncryptionByDefault",
    ],
    "myapp.cloudshell": [
        "CreateSession",
        "DeleteSession",
        "RedeemCode",
        "PutCredentials",
        "GetEnvironmentStatus",
        "SendHeartBeat",
        "DescribeEnvironments",
    ],
    # Add other mappings as needed
}

def lambda_handler(event, context):
    try:
        # Extract 'num_events' from the incoming event; default to 10 if not provided
        num_events = event.get("num_events", 10)
        events = []
        logger.info(f"Generating {num_events} event(s).")

        for _ in range(num_events):
            event_name = random.choice(EVENT_NAMES)
            source = next(
                (src for src, events_list in EVENT_SOURCES.items() if event_name in events_list),
                "myapp.unknown"
            )

            # Prepare the event detail
            detail = {
                "eventName": event_name,
                "timestamp": datetime.utcnow().isoformat(),
                "metadata": {
                    "requestId": context.aws_request_id if context else "unknown",
                    "region": "us-east-1",
                },
            }

            # Append the event to the batch
            events.append({
                "EventBusName": EVENT_BUS_NAME,
                "Source": source,
                "DetailType": "AWS API Call via EventEmitterFunction",
                "Detail": json.dumps(detail),
            })

        # Generate confidence scores for each event
        confidence_scores = [round(random.uniform(0.7, 0.99), 2) for _ in range(num_events)]

        # Function to split events into chunks of 10
        def chunks(lst, n):
            """Yield successive n-sized chunks from lst."""
            for i in range(0, len(lst), n):
                yield lst[i:i + n]

        # Send events to EventBridge in batches of 10
        all_responses = []
        for batch in chunks(events, 10):
            response = eventbridge.put_events(Entries=batch)
            logger.info(f"Sent {len(batch)} event(s) to EventBridge. Response: {response}")
            all_responses.append(response)

        # Check for failed entries
        failed_entries = []
        for response in all_responses:
            failed = response.get('FailedEntryCount', 0)
            if failed > 0:
                # Extract the failed entries
                failed_entries.extend(response.get('Entries', [])[:failed])

        if failed_entries:
            logger.error(f"Failed to send {len(failed_entries)} event(s) to EventBridge.")
            return {
                "statusCode": 500,
                "error": f"Failed to send {len(failed_entries)} event(s) to EventBridge.",
                "failed_events": [
                    {
                        "eventName": json.loads(event["Detail"])["eventName"],
                        "predicted_source": event["Source"],
                        "detail": event["Detail"]
                    }
                    for event in failed_entries
                ],
            }

        # Return emitted events with their predicted sources and confidence scores
        return {
            "statusCode": 200,
            "message": f"Successfully emitted {num_events} events.",
            "events": [
                {
                    "eventName": json.loads(event["Detail"])["eventName"],
                    "predicted_source": event["Source"],
                    "score": confidence_scores[i]
                }
                for i, event in enumerate(events)
            ],
        }

    except Exception as e:
        logger.error(f"Error emitting events: {str(e)}", exc_info=True)
        return {
            "statusCode": 500,
            "error": str(e),
        }
